In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytickersymbols import PyTickerSymbols

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# Calculate RSI manually using pandas
def calculate_rsi(data, period=7):
    delta = data['Adj Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

In [ ]:
# Implement the trading strategy with stop-sell condition
def trading_strategy(data):
    position = 0  # 1 means we own the stock, 0 means we don't
    holding_period = 0  # Track how long we’ve held the position
    max_holding_days = 7  # Stop-sell after 7 days
    buy_price = 0
    data['Signal'] = 0

    for i in range(1, len(data)):
        if data['RSI'][i-1] < 90 and data['RSI'][i] >= 90 and position == 0:
            # Buy signal: RSI crosses from below 90 to above 90
            position = 1
            holding_period = 1  # Reset holding period counter on buy
            buy_price = data['Adj Close'][i]
            data['Signal'][i] = 1  # 1 for buy
            #print(f"Bought at {buy_price} on {data.index[i]}")
        
        elif position == 1:
            holding_period += 1
            
            # Sell signal: RSI crosses from above 10 to below 10 or stop-sell after 7 days
            if (data['RSI'][i-1] > 10 and data['RSI'][i] <= 10) or holding_period > max_holding_days:
                position = 0
                sell_price = data['Adj Close'][i]
                data['Signal'][i] = -1  # -1 for sell
                #print(f"Sold at {sell_price} on {data.index[i]} (held for {holding_period - 1} days)")
                holding_period = 0  # Reset the holding period after selling
    
    return data

In [ ]:
# Backtest the strategy
def backtest(data):
    # Create Position column where we hold the stock only between buy and sell signals
    data['Position'] = 0  # Initialize the position as no stock held

    for i in range(1, len(data)):
        if data['Signal'][i] == 1:  # Buy signal
            data['Position'][i] = 1
        elif data['Signal'][i] == -1:  # Sell signal
            data['Position'][i] = 0
        else:
            data['Position'][i] = data['Position'][i-1]  # Continue holding or not

    # Calculate daily returns
    data['Daily Return'] = data['Adj Close'].pct_change()
    
    # Strategy returns only when we're in position
    data['Strategy Return'] = data['Daily Return'] * data['Position'].shift(1)
    
    # Set returns to 0 for days when not holding a position
    data['Strategy Return'].fillna(0, inplace=True)

    # Calculate cumulative returns
    data['Cumulative Market Return'] = (1 + data['Daily Return']).cumprod()
    data['Cumulative Strategy Return'] = (1 + data['Strategy Return']).cumprod()
    
    return data

In [ ]:
def backtest_portfolio(symbols, period):
    portfolio_returns = pd.DataFrame()

    pulled_data = yf.download(are tsymbols, period = period, interval = "1d", group_by="ticker") #Pulling all the data for all the stocks
    
    for symbol in symbols:
        #print(f"Processing {symbol}...")
        try:
            individual_stock_data = pulled_data[symbol]
            if len(individual_stock_data) == 0:  # Skip symbols with no data
                continue
            
            individual_stock_data = calculate_rsi(individual_stock_data)
            individual_stock_data = trading_strategy(individual_stock_data)
            individual_stock_data = backtest(individual_stock_data)
            
            # Store strategy return for each stock
            portfolio_returns[symbol] = individual_stock_data['Strategy Return']
        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    # Average the returns of all stocks to simulate equal weighting
    portfolio_returns['Portfolio Return'] = portfolio_returns.mean(axis=1)
    
    # Calculate cumulative returns for the portfolio
    portfolio_returns['Cumulative Portfolio Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod()

    total_strategy_return = (portfolio_returns['Cumulative Portfolio Return'].iloc[-1] - 1)*100
    total_strategy_return = "{:.2f}%".format(total_strategy_return)

    trading_days_per_year = 252
    trading_days_per_month = trading_days_per_year // 12

    annual_risk_free_rate=0.045
    
    if period == "10y":
        days = 10 * trading_days_per_year
        risk_free_rate = (1+ annual_risk_free_rate)**10 - 1
    elif period == "5y":
        days = 5 * trading_days_per_year
        risk_free_rate = (1+ annual_risk_free_rate)**5 - 1
    elif period == "2y":
        days = 2 * trading_days_per_year
        risk_free_rate = (1+ annual_risk_free_rate)**2 - 1
    elif period == "1y":
        days = trading_days_per_year
        risk_free_rate = annual_risk_free_rate
    elif period == "6mo":
        days = 6 * trading_days_per_month
        risk_free_rate = (1+ annual_risk_free_rate)**(6/12) - 1
    elif period == "3mo":
        days = 3 * trading_days_per_month
        risk_free_rate = (1+ annual_risk_free_rate)**(3/12) - 1
    elif period == "1mo":
        days = trading_days_per_month
        risk_free_rate = (1+ annual_risk_free_rate)**(1/12) - 1

    #Sharpe Ratio
    annualized_return = ((1 + portfolio_returns['Portfolio Return'].mean()) ** days) - 1
    annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(days)
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

    return total_strategy_return, sharpe_ratio

In [ ]:
def plot_portfolio_results(portfolio_returns):
    plt.figure(figsize=(14, 7))
    plt.plot(portfolio_returns['Cumulative Portfolio Return'], label='Cumulative Portfolio Return')
    plt.title('Portfolio Performance (Equal Weighting)')
    plt.legend()
    plt.show()

In [ ]:
def main_portfolio():
    stock_data = PyTickerSymbols()
    sp500_symbols = stock_data.get_stocks_by_index('S&P 500')
    symbols = [stock['symbol'] for stock in sp500_symbols]  # S&P 500 stock symbols
    #symbols = ["NVDA", "AAPL", "MSTR"]

    #periods = ["10y", "5y", "2y", "1y", "6mo", "3mo", "1mo"] 
    periods = ["3mo", "1mo"] 

    #df = pd.DataFrame(columns=["10y", "5y", "2y", "1y", "6mo", "3mo", "1mo"])
    df = pd.DataFrame(columns=["3mo", "1mo"])

    returns = {}
    sharpe = {}

    for period in periods:
        # Backtest the portfolio
        total_return, sharpe_ratio = backtest_portfolio(symbols, period)
        returns[period] = total_return
        sharpe[period] = sharpe_ratio

    df = pd.concat([df, pd.DataFrame([returns])], ignore_index=True)
    df = pd.concat([df, pd.DataFrame([sharpe])], ignore_index=True)

    df.index = ['Portfolio Returns', 'Sharpe Ratio']
        
    return df


In [ ]:
df = main_portfolio()
df

In [ ]:
df.to_csv('s&p500 7 day buy90 sell10 part 3.csv', index=True)